In [3]:
import pandas as pd

data_D_U_train = pd.read_csv('../dataset/train_D_U.csv')
data_D_U_test = pd.read_csv('../dataset/test_D_U.csv')

data_D_U_O_train = pd.read_csv('../dataset/train_D_U_O.csv')
data_D_U_O_test = pd.read_csv('../dataset/test_D_U_O.csv')

### DU

In [15]:
X_train_DU = data_D_U_train[data_D_U_train.columns.difference(['outcome_critical','Unnamed: 0'])]
X_test_DU = data_D_U_test[data_D_U_test.columns.difference(['outcome_critical','Unnamed: 0'])]

y_train_DU = data_D_U_train['outcome_critical']
y_test_DU = data_D_U_test['outcome_critical']

### DUO

In [20]:
X_train_DUO = data_D_U_O_train[data_D_U_O_train.columns.difference(['outcome_critical','Unnamed: 0'])]
X_test_DUO = data_D_U_O_test[data_D_U_O_test.columns.difference(['outcome_critical','Unnamed: 0'])]

y_train_DUO = data_D_U_O_train['outcome_critical']
y_test_DUO = data_D_U_O_test['outcome_critical']

### Random forest

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

rand_clf = RandomForestClassifier(criterion='entropy', bootstrap=True, random_state=2022)

### DU

In [19]:
rand_clf.fit(X_train_DU, y_train_DU)
y_pred = rand_clf.predict(X_test_DU)

print('훈련세트 정확도: {:.3f}' .format(rand_clf.score(X_train_DU, y_train_DU)))
print('테스트세트 정확도: {:.3f}' .format(rand_clf.score(X_test_DU, y_test_DU)))

훈련세트 정확도: 1.000
테스트세트 정확도: 0.852


### DUO

In [21]:
rand_clf.fit(X_train_DUO, y_train_DUO)
y_pred = rand_clf.predict(X_test_DUO)

print('훈련세트 정확도: {:.3f}' .format(rand_clf.score(X_train_DUO, y_train_DUO)))
print('테스트세트 정확도: {:.3f}' .format(rand_clf.score(X_test_DUO, y_test_DUO)))

훈련세트 정확도: 1.000
테스트세트 정확도: 0.884


### lightGBM

In [12]:
import lightgbm as lgb

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def setting(y_pred):
    for i in range(len(y_pred)):
        if y_pred[i]>=.5:       # setting threshold to .5
           y_pred[i]=1
        else:  
           y_pred[i]=0

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0: .4f}, 정밀도: {1: .4f}, 재현율: {2: .4f},\nF1: {3: .4f}, AUC: {4: .4f}'.format(accuracy, precision, recall, f1, roc_auc))

### DU

In [16]:
model_class = lgb.LGBMClassifier(boosting='goss', learning_rate=0.08, num_iterations=500, early_stopping_rounds=50,
                                 max_depth=-1,random_state=42, feature_fraction=0.8, bagging_fraction=0.7,
                                 is_training_metric=True, num_leaves=25, objective='binary', metric='bagging_fraction')
model_class.fit(X_train_DU,y_train_DU,eval_set=[(X_test_DU,y_test_DU),(X_train_DU,y_train_DU)], verbose=20,eval_metric='logloss')

y_pred_class = model_class.predict(X_test_DU)

/Users/son/opt/anaconda3/envs/machine_learning/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/son/opt/anaconda3/envs/machine_learning/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 50 rounds
[20]	training's binary_logloss: 0.332974	valid_0's binary_logloss: 0.351506
[40]	training's binary_logloss: 0.270213	valid_0's binary_logloss: 0.2966
[60]	training's binary_logloss: 0.250652	valid_0's binary_logloss: 0.281933
[80]	training's binary_logloss: 0.241652	valid_0's binary_logloss: 0.278455
[100]	training's binary_logloss: 0.235324	valid_0's binary_logloss: 0.276557
[120]	training's binary_logloss: 0.230319	valid_0's binary_logloss: 0.277047
[140]	training's binary_logloss: 0.225385	valid_0's binary_logloss: 0.278436
Early stopping, best iteration is:
[92]	training's binary_logloss: 0.237874	valid_0's binary_logloss: 0.276414


In [18]:
setting(y_pred_class)
get_clf_eval(y_test_DU, y_pred_class)

오차 행렬
[[71797 12102]
 [  573  5323]]
정확도:  0.8588, 정밀도:  0.3055, 재현율:  0.9028,
F1:  0.4565, AUC:  0.8793


### DUO

In [24]:
model_class = lgb.LGBMClassifier(boosting='goss', learning_rate=0.08, num_iterations=500, early_stopping_rounds=50,
                                 max_depth=-1,random_state=42, feature_fraction=0.8, bagging_fraction=0.7,
                                 is_training_metric=True, num_leaves=25, objective='binary', metric='bagging_fraction')
model_class.fit(X_train_DUO,y_train_DUO,eval_set=[(X_test_DUO,y_test_DUO),(X_train_DUO,y_train_DUO)], verbose=20,eval_metric='logloss')

y_pred_class = model_class.predict(X_test_DUO)

/Users/son/opt/anaconda3/envs/machine_learning/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/son/opt/anaconda3/envs/machine_learning/lib/python3.8/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
Training until validation scores don't improve for 50 rounds
[20]	training's binary_logloss: 0.302907	valid_0's binary_logloss: 0.321136
[40]	training's binary_logloss: 0.234059	valid_0's binary_logloss: 0.256281
[60]	training's binary_logloss: 0.21107	valid_0's binary_logloss: 0.23395
[80]	training's binary_logloss: 0.201196	valid_0's binary_logloss: 0.226128
[100]	training's binary_logloss: 0.194739	valid_0's binary_logloss: 0.221034
[120]	training's binary_logloss: 0.190085	valid_0's binary_logloss: 0.218739


In [25]:
setting(y_pred_class)
get_clf_eval(y_test_DUO, y_pred_class)

오차 행렬
[[75710  8189]
 [  924  4972]]
정확도:  0.8985, 정밀도:  0.3778, 재현율:  0.8433,
F1:  0.5218, AUC:  0.8728
